In [1]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer
from pymongo import MongoClient
from pymongo import GEOSPHERE
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from folium import CircleMarker
import numpy as py
load_dotenv()

True

In [2]:
client = MongoClient("localhost:27017")
db = client["Ironhack"] # I or i 
client_id= os.getenv("CLIENT_ID")
client_s= os.getenv("CLIENT_SECRET")
client_key=os.getenv("API_KEY")

In [3]:
# At first the choice was made to focus on Santa Monica since this is where GTA takes places and hosts host of gamining companies
initial_lat = 34.0522
initial_long = -118.2437


In [4]:
LA_map = folium.Map(location = [initial_lat, initial_long], zoom_start=9.5)
LA_map

In [5]:
companies = db.get_collection('companies') # C or c

### function for creating the dataframe of all the info needed

In [6]:
# function for creating the dataframe of all the info needed

# lets drop some more, one of the most important requiremetns is having design companies nearby.
# We should remove all comnaies that are not design companies 

def Company_in_cali(c):
    filter_ = {'$and':[{'offices': {'$exists': 1}},{'offices.latitude':{'$type':1}},{'offices.longitude':{'$type': 1}}]}
    projection = {"_id":0,'name':1,'founded_year':1,'offices.country_code':1,"offices.state_code":1,'offices.city':1,'offices.address1':1,'offices.address2':1,'offices.latitude':1,'offices.longitude':1}
    sorting = {'offices.country_code':-1}
    lis = list(companies.find(filter_, projection).sort("offices.country_code", -1))
    df = pd.DataFrame(lis).explode("offices").reset_index(drop=True)
    df = pd.concat([df, df["offices"].apply(pd.Series)], axis=1).reset_index(drop=True)
    df.dropna(subset=["latitude"],inplace=True)
    df.dropna(subset=["founded_year"],inplace=True)
    df.dropna(subset=["address1"],inplace=True)
    df.dropna(subset=["address2"],inplace=True)
    df.dropna(subset=["city"],inplace=True)
    newcols = [element for element in df.columns if element != 'offices']
    df=df[newcols].reset_index(drop=True)
    df=df[df["country_code"]=="USA"]
    df=df[df['state_code'] == 'CA'].reset_index(drop=True)
    df = df.drop(columns="founded_year")
    return df


In [7]:
df= Company_in_cali(companies)
df

,name,address1,address2,city,state_code,country_code,latitude,longitude
0,Yola,201 Mission Street,Suite 2250,San Francisco,CA,USA,37.791115,-122.395475
1,Buongiorno,485 Alberto Way,Suite 210,Los Gatos,CA,USA,37.229898,-121.971853
2,InMobi,475 Brannan,Suite 420,San Francisco,CA,USA,37.446550,-122.170921
3,LinQpay,,,Carlsbad,CA,USA,33.095680,-117.244034
4,Sparkpr,2 Bryant Street,Suite 100,San Francisco,CA,USA,37.787394,-122.388442
...,...,...,...,...,...,...,...,...
2396,Cantimer,3700 Haven Court,,Menlo Park,CA,USA,37.484616,-122.181753
2397,Celestial Semiconductor,"2635 North 1st Street, Suite 150",,San Jose,CA,USA,37.386417,-121.929170
2398,Purfresh,"1350 Willow Road,",Suite 102,Menlo Park,CA,USA,37.477008,-122.151599
2399,MyChances,,,Menlo Park,CA,USA,37.448491,-122.180281


## Dataframe of all the employees and their count and salaries

- 20 Designers
- 5 UI/UX Engineers
- 10 Frontend Developers
- 15 Data Engineers
- 5 Backend Developers
- 20 Account Managers
- 1 Maintenance guy that loves basketball
- 10 Executives
- 1 CEO/President.

In [8]:
# Dataframe of all the employees and their count and salaries

employee_table = {'Title': ['Game designer', 'UX/UI', 'Front-end developer', 'Data engineer', 'Back-end developer',
                      'Account manager', 'Maintenance guy', 'Executive', 'CEO' ],
                'Salary': [115568, 104266, 87240, 112493, 80258, 58670, 40482, 52522, 148598],
                'Count': [20, 5, 10, 15, 5, 20, 1, 10,1]
                 } 
# We can sue this to weigh our decision, Dani did something like this,
# I think its good for reasoning why something is more important than another
# salaries are from glassdoor and are average salaries 

df_employee_table = pd.DataFrame(employee_table).explode("Title")
df_employee_table['total spending'] = df_employee_table.eval('Salary*Count')
df_employee_table.sort_values(by=['total spending'], ascending=False)

,Title,Salary,Count,total spending
0,Game designer,115568,20,2311360
3,Data engineer,112493,15,1687395
5,Account manager,58670,20,1173400
2,Front-end developer,87240,10,872400
7,Executive,52522,10,525220
1,UX/UI,104266,5,521330
4,Back-end developer,80258,5,401290
8,CEO,148598,1,148598
6,Maintenance guy,40482,1,40482


## Heatmapping function

In [9]:

def Heat_map_cali(df,radius):
    m = folium.Map(location = [34.0218593,-118.498265], zoom_start=10)
    HeatMap(data=df[["latitude","longitude"]],radius=radius).add_to(m)
    return m


In [10]:
Heat_map_cali(df,15)

## Foursquare api Work soo far


In [11]:
query = {"offices.city": "Los Angeles"}
projection = {"_id":0,"name":1,"offices.city":1, "country_code":1,
              "offices.latitude":1, "offices.longitude":1,"office.city":1}

list(companies.find(query, projection))

[{'name': 'Helio',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.057498,
    'longitude': -118.446596}]},
 {'name': 'Stickam',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.051409,
    'longitude': -118.254558}]},
 {'name': 'Ustream',
  'offices': [{'city': 'San Francisco',
    'latitude': 37.392936,
    'longitude': -122.07948},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None},
   {'city': 'Budapest', 'latitude': None, 'longitude': None}]},
 {'name': 'TripUp',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.052187,
    'longitude': -118.243425}]},
 {'name': 'Eventful',
  'offices': [{'city': 'San Diego',
    'latitude': 32.872081,
    'longitude': -117.206292},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None}]},
 {'name': 'ScanScout',
  'offices': [{'city': 'Boston',
    'latitude': 42.340889,
    'longitude': -71.051931},
   {'city': 'New York', 'latitude': 40.739833, 'longitude': -73.993158},
   {'city': 'Los Angeles', 'la

In [12]:
len(list(companies.find(query, projection)))

219

In [13]:
LosAngeles = list(companies.find(query, projection))

In [14]:
LosAngeles

[{'name': 'Helio',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.057498,
    'longitude': -118.446596}]},
 {'name': 'Stickam',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.051409,
    'longitude': -118.254558}]},
 {'name': 'Ustream',
  'offices': [{'city': 'San Francisco',
    'latitude': 37.392936,
    'longitude': -122.07948},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None},
   {'city': 'Budapest', 'latitude': None, 'longitude': None}]},
 {'name': 'TripUp',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.052187,
    'longitude': -118.243425}]},
 {'name': 'Eventful',
  'offices': [{'city': 'San Diego',
    'latitude': 32.872081,
    'longitude': -117.206292},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None}]},
 {'name': 'ScanScout',
  'offices': [{'city': 'Boston',
    'latitude': 42.340889,
    'longitude': -71.051931},
   {'city': 'New York', 'latitude': 40.739833, 'longitude': -73.993158},
   {'city': 'Los Angeles', 'la

In [15]:
LosAngeles_new = []
for i in LosAngeles:
    for j in i['offices']:
        if j['city'] == 'Los Angeles':
            city = j['city']
            lat = j['latitude']
            lon = j['longitude']
            name = i['name']
            type_ = ({"typepoint": 
                      {"type": "Point", 
                         "coordinates": [j['latitude'], j['longitude']]}})
            
            LosAngeles_new.append({"name":name, "lat":lat, "long":lon, "type":type_})
LosAngeles_new

[{'name': 'Helio',
  'lat': 34.057498,
  'long': -118.446596,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.057498, -118.446596]}}},
 {'name': 'Stickam',
  'lat': 34.051409,
  'long': -118.254558,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.051409, -118.254558]}}},
 {'name': 'Ustream',
  'lat': None,
  'long': None,
  'type': {'typepoint': {'type': 'Point', 'coordinates': [None, None]}}},
 {'name': 'TripUp',
  'lat': 34.052187,
  'long': -118.243425,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.052187, -118.243425]}}},
 {'name': 'Eventful',
  'lat': None,
  'long': None,
  'type': {'typepoint': {'type': 'Point', 'coordinates': [None, None]}}},
 {'name': 'ScanScout',
  'lat': 34.050887,
  'long': -118.459995,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.050887, -118.459995]}}},
 {'name': 'WebbAlert',
  'lat': 34.052187,
  'long': -118.243425,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.0

In [16]:
len(LosAngeles_new)

220

In [17]:
df_LosAngeles = pd.DataFrame(LosAngeles_new)
df_LosAngeles

,name,lat,long,type
0,Helio,34.057498,-118.446596,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Stickam,34.051409,-118.254558,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,Ustream,NaN,NaN,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,TripUp,34.052187,-118.243425,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,Eventful,NaN,NaN,"{'typepoint': {'type': 'Point', 'coordinates':..."
...,...,...,...,...
215,California 7 Studios,NaN,NaN,"{'typepoint': {'type': 'Point', 'coordinates':..."
216,Pipit Interactive,34.045499,-118.470360,"{'typepoint': {'type': 'Point', 'coordinates':..."
217,Top Rated Website Designers,33.993326,-118.398784,"{'typepoint': {'type': 'Point', 'coordinates':..."
218,SOA Software,NaN,NaN,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [18]:
df_LosAngeles.isna().sum()

name     0
lat     52
long    52
type     0
dtype: int64

In [19]:
df_LosAngeles_copy = df_LosAngeles.copy()

In [20]:
df_LosAngeles_copy.dropna(subset=['lat', 'long'],inplace=True)

In [21]:
df_LosAngeles_copy.isna().sum()

name    0
lat     0
long    0
type    0
dtype: int64

In [22]:
len(df_LosAngeles_copy)

168

In [23]:
df_LosAngeles_copy = df_LosAngeles_copy.reset_index(drop=True)


In [24]:
df_LosAngeles_copy

,name,lat,long,type
0,Helio,34.057498,-118.446596,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Stickam,34.051409,-118.254558,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,TripUp,34.052187,-118.243425,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,ScanScout,34.050887,-118.459995,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,WebbAlert,34.052187,-118.243425,"{'typepoint': {'type': 'Point', 'coordinates':..."
...,...,...,...,...
163,AECOM Technology,34.050796,-118.256861,"{'typepoint': {'type': 'Point', 'coordinates':..."
164,Culture Jam,34.013715,-118.466060,"{'typepoint': {'type': 'Point', 'coordinates':..."
165,Pipit Interactive,34.045499,-118.470360,"{'typepoint': {'type': 'Point', 'coordinates':..."
166,Top Rated Website Designers,33.993326,-118.398784,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [25]:
LA_group = folium.FeatureGroup(name="Los Angeles")

In [26]:
HeatMap(data=df_LosAngeles_copy[["lat", "long"]], radius=15,
       gradient = {0.5: 'blue', 0.5:'green', 0.65: 'yellow', 1: 'red'}).add_to(LA_group)

In [27]:
LA_group.add_to(LA_map)


In [28]:
LA_map

In [29]:
SM_lat = 34.0195
SM_long = -118.4912

In [30]:
SM_map = folium.Map(location = [SM_lat, SM_long], zoom_start=10)
folium.Marker([SM_lat, SM_long]).add_to(SM_map)
"""folium.Circle([SM_lat, SM_long], radius=500).add_to(SM_map)
folium.Circle([SM_lat, SM_long], radius=1000).add_to(SM_map)
folium.Circle([SM_lat, SM_long], radius=3000).add_to(SM_map)"""

'folium.Circle([SM_lat, SM_long], radius=500).add_to(SM_map)\nfolium.Circle([SM_lat, SM_long], radius=1000).add_to(SM_map)\nfolium.Circle([SM_lat, SM_long], radius=3000).add_to(SM_map)'

In [31]:
SM_map

In [32]:
SM_lat_updated = 34.03713832766421
SM_long_updated = -118.46959297037199

In [33]:
SM_map_updated = folium.Map(location = [SM_lat_updated, SM_long_updated], zoom_start=10)
folium.Marker([SM_lat_updated, SM_long_updated]).add_to(SM_map)
"""folium.Circle([SM_lat_updated, SM_long_updated], radius=500, color="red").add_to(SM_map_updated)
folium.Circle([SM_lat_updated, SM_long_updated], radius=1000, color="yellow").add_to(SM_map_updated)
folium.Circle([SM_lat_updated, SM_long_updated], radius=2000, color="green").add_to(SM_map_updated)
folium.Circle([SM_lat_updated, SM_long_updated], radius=3000, color="blue").add_to(SM_map_updated)"""

'folium.Circle([SM_lat_updated, SM_long_updated], radius=500, color="red").add_to(SM_map_updated)\nfolium.Circle([SM_lat_updated, SM_long_updated], radius=1000, color="yellow").add_to(SM_map_updated)\nfolium.Circle([SM_lat_updated, SM_long_updated], radius=2000, color="green").add_to(SM_map_updated)\nfolium.Circle([SM_lat_updated, SM_long_updated], radius=3000, color="blue").add_to(SM_map_updated)'

In [34]:
SM_map_updated

In [35]:
def querySM(query, location, radius):

    if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")
    else:
           query_ok = query
    url = f"https://api.foursquare.com/v3/places/search?query={query_ok}&ll={str(location[0]).strip()}%2C{str(location[1]).strip()}&radius={radius}&limit=50"
    headers =  {"Accept": "application/json","Authorization":  f"{client_key}"
    }
    response = requests.get(url, headers=headers)
    
    if response:
        response=response.json()['results']
        places_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, lon]}}} for i in response]
        return places_list

In [36]:
def queryAirport(query, location, radius):

    if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")
    else:
           query_ok = query
    url = f"https://api.foursquare.com/v3/places/search?query={query_ok}&ll={str(location[0]).strip()}%2C{str(location[1]).strip()}&radius={radius}&categories=19040&limit=50"
    headers =  {"Accept": "application/json","Authorization":  f"{client_key}"
    }
    response = requests.get(url, headers=headers)
    
    if response:
        response=response.json()['results']
        places_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, lon]}}} for i in response]
        return places_list
    


In [37]:
def queryStadium(query, location, radius):

    if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")
    else:
           query_ok = query
    url = f"https://api.foursquare.com/v3/places/search?query={query_ok}&ll={str(location[0]).strip()}%2C{str(location[1]).strip()}&radius={radius}&limit=1"
    headers =  {"Accept": "application/json","Authorization":  f"{client_key}"
    }
    response = requests.get(url, headers=headers)
    
    if response:
        response=response.json()['results']
        places_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, lon]}}} for i in response]
        return places_list

In [38]:
# Why is noting being save into the dataframe?
 # we have tried differend radius, Something clearly happens here as this is where loading gets slow


In [39]:
def C_C(col,q,r,df):
    lis=[]
    for i in range(len(df)):
        if querySM(q,[df.iloc[i][1], df.iloc[i][2]],r):
            lis.append(querySM(q,[df.iloc[i][1], df.iloc[i][2]],r))
        else:
            lis.append(py.nan)
    dic ={col:lis}
    dff= pd.DataFrame.from_dict(dic)
    return dff

In [40]:
def C_A(col,q,r,df):
    lis=[]
    for i in range(len(df)):
        if queryAirport(q,[df.iloc[i][1], df.iloc[i][2]],r):
            lis.append(queryAirport(q,[df.iloc[i][1], df.iloc[i][2]],r))
        else:
            lis.append(py.nan)
    dic ={col:lis}
    dff= pd.DataFrame.from_dict(dic)
    return dff

In [41]:
def C_S(col,q,r,df):
    lis=[]
    for i in range(len(df)):
        if queryStadium(q,[df.iloc[i][1], df.iloc[i][2]],r):
            lis.append(queryStadium(q,[df.iloc[i][1], df.iloc[i][2]],r))
        else:
            lis.append(py.nan)
    dic ={col:lis}
    dff= pd.DataFrame.from_dict(dic)
    return dff

In [42]:
dfn = df_LosAngeles_copy.copy()
dfu= C_C("Design","Design",300,dfn)
dfu.isnull().sum()

Design    130
dtype: int64

In [43]:
df_LosAngeles_copy = pd.concat([df_LosAngeles_copy,dfu],axis=1)

In [44]:
df_LosAngeles_copy.dropna(inplace=True)

In [45]:
df_LosAngeles_copy.reset_index(drop=True,inplace=True)

In [46]:
df_LosAngeles_copy

,name,lat,long,type,Design
0,Social Platform,34.101617,-118.327071,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'No Such Agency', 'lat': 34.100342, ..."
1,Jirbo,34.064629,-118.469316,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'A Stitch In Time', 'lat': 34.066449..."
2,JuicyCampus,33.954874,-118.396507,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Fastframe, Expert Picture Framing',..."
3,Napster,34.080772,-118.389196,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Furstenberg Alexandra Von', 'lat': ..."
4,SEO Intelligence,34.041037,-118.233410,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Nowness I Dazed & Confused', 'lat':..."
5,DailyStrength,34.043948,-118.468204,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Designers Call', 'lat': 34.045491, ..."
6,CrewedUp,34.089679,-118.292618,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Phen Media', 'lat': 34.091487, 'lon..."
7,FASTALLEY,34.081952,-118.357658,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Alex Tailor', 'lat': 34.083589, 'lo..."
8,Wedding Channel,34.048681,-118.258778,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Design Digital', 'lat': 34.048574, ..."
9,Causecast,34.018349,-118.486889,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Langley Productions', 'lat': 34.020..."


In [47]:
dfn = df_LosAngeles_copy.copy()
dfu= C_C("school","school",300,dfn)
dfu.isnull().sum()

school    2
dtype: int64

In [48]:
dfu

,school
0,"[{'name': 'LCG Footer', 'lat': 34.101301, 'lon..."
1,"[{'name': 'St Matthews School', 'lat': 34.0667..."
2,"[{'name': 'Kumon Math and Reading Center', 'la..."
3,"[{'name': 'Johannes School Of Music', 'lat': 3..."
4,"[{'name': 'Lock and Key Productions', 'lat': 3..."
5,"[{'name': 'Cotsen Family Foundation', 'lat': 3..."
6,"[{'name': 'Spiritual Arts', 'lat': 34.089903, ..."
7,"[{'name': 'Westside Community Adult School', '..."
8,[{'name': 'Alliance College-Ready Public Schoo...
9,"[{'name': 'TeamNow', 'lat': 34.020013, 'long':..."


In [49]:
df_LosAngeles_copy = pd.concat([df_LosAngeles_copy,dfu],axis=1)
df_LosAngeles_copy.dropna(inplace=True)
df_LosAngeles_copy.reset_index(drop=True,inplace=True)
df_LosAngeles_copy

,name,lat,long,type,Design,school
0,Social Platform,34.101617,-118.327071,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'No Such Agency', 'lat': 34.100342, ...","[{'name': 'LCG Footer', 'lat': 34.101301, 'lon..."
1,Jirbo,34.064629,-118.469316,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'A Stitch In Time', 'lat': 34.066449...","[{'name': 'St Matthews School', 'lat': 34.0667..."
2,JuicyCampus,33.954874,-118.396507,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Fastframe, Expert Picture Framing',...","[{'name': 'Kumon Math and Reading Center', 'la..."
3,Napster,34.080772,-118.389196,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Furstenberg Alexandra Von', 'lat': ...","[{'name': 'Johannes School Of Music', 'lat': 3..."
4,SEO Intelligence,34.041037,-118.233410,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Nowness I Dazed & Confused', 'lat':...","[{'name': 'Lock and Key Productions', 'lat': 3..."
5,DailyStrength,34.043948,-118.468204,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Designers Call', 'lat': 34.045491, ...","[{'name': 'Cotsen Family Foundation', 'lat': 3..."
6,CrewedUp,34.089679,-118.292618,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Phen Media', 'lat': 34.091487, 'lon...","[{'name': 'Spiritual Arts', 'lat': 34.089903, ..."
7,FASTALLEY,34.081952,-118.357658,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Alex Tailor', 'lat': 34.083589, 'lo...","[{'name': 'Westside Community Adult School', '..."
8,Wedding Channel,34.048681,-118.258778,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Design Digital', 'lat': 34.048574, ...",[{'name': 'Alliance College-Ready Public Schoo...
9,Causecast,34.018349,-118.486889,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Langley Productions', 'lat': 34.020...","[{'name': 'TeamNow', 'lat': 34.020013, 'long':..."


In [50]:
len(df_LosAngeles_copy)

36

In [51]:
dfn = df_LosAngeles_copy.copy()
dfu= C_C("Clubs","club",500,dfn)
dfu.isnull().sum()

Clubs    1
dtype: int64

In [52]:
df_LosAngeles_copy

,name,lat,long,type,Design,school
0,Social Platform,34.101617,-118.327071,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'No Such Agency', 'lat': 34.100342, ...","[{'name': 'LCG Footer', 'lat': 34.101301, 'lon..."
1,Jirbo,34.064629,-118.469316,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'A Stitch In Time', 'lat': 34.066449...","[{'name': 'St Matthews School', 'lat': 34.0667..."
2,JuicyCampus,33.954874,-118.396507,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Fastframe, Expert Picture Framing',...","[{'name': 'Kumon Math and Reading Center', 'la..."
3,Napster,34.080772,-118.389196,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Furstenberg Alexandra Von', 'lat': ...","[{'name': 'Johannes School Of Music', 'lat': 3..."
4,SEO Intelligence,34.041037,-118.233410,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Nowness I Dazed & Confused', 'lat':...","[{'name': 'Lock and Key Productions', 'lat': 3..."
5,DailyStrength,34.043948,-118.468204,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Designers Call', 'lat': 34.045491, ...","[{'name': 'Cotsen Family Foundation', 'lat': 3..."
6,CrewedUp,34.089679,-118.292618,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Phen Media', 'lat': 34.091487, 'lon...","[{'name': 'Spiritual Arts', 'lat': 34.089903, ..."
7,FASTALLEY,34.081952,-118.357658,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Alex Tailor', 'lat': 34.083589, 'lo...","[{'name': 'Westside Community Adult School', '..."
8,Wedding Channel,34.048681,-118.258778,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Design Digital', 'lat': 34.048574, ...",[{'name': 'Alliance College-Ready Public Schoo...
9,Causecast,34.018349,-118.486889,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Langley Productions', 'lat': 34.020...","[{'name': 'TeamNow', 'lat': 34.020013, 'long':..."


In [53]:
df_LosAngeles_copy = pd.concat([df_LosAngeles_copy,dfu],axis=1)
df_LosAngeles_copy.dropna(inplace=True)
df_LosAngeles_copy.reset_index(drop=True,inplace=True)
df_LosAngeles_copy

,name,lat,long,type,Design,school,Clubs
0,Social Platform,34.101617,-118.327071,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'No Such Agency', 'lat': 34.100342, ...","[{'name': 'LCG Footer', 'lat': 34.101301, 'lon...","[{'name': 'H Club Los Angeles', 'lat': 34.1023..."
1,JuicyCampus,33.954874,-118.396507,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Fastframe, Expert Picture Framing',...","[{'name': 'Kumon Math and Reading Center', 'la...",[{'name': 'Rotary International District 5280'...
2,Napster,34.080772,-118.389196,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Furstenberg Alexandra Von', 'lat': ...","[{'name': 'Johannes School Of Music', 'lat': 3...","[{'name': 'Juicy', 'lat': 34.082583, 'long': -..."
3,SEO Intelligence,34.041037,-118.233410,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Nowness I Dazed & Confused', 'lat':...","[{'name': 'Lock and Key Productions', 'lat': 3...","[{'name': 'Los Angeles Gun Club', 'lat': 34.03..."
4,DailyStrength,34.043948,-118.468204,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Designers Call', 'lat': 34.045491, ...","[{'name': 'Cotsen Family Foundation', 'lat': 3...","[{'name': 'Water Buffalo Club', 'lat': 34.0439..."
5,CrewedUp,34.089679,-118.292618,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Phen Media', 'lat': 34.091487, 'lon...","[{'name': 'Spiritual Arts', 'lat': 34.089903, ...","[{'name': 'The Vermont Hollywood', 'lat': 34.0..."
6,FASTALLEY,34.081952,-118.357658,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Alex Tailor', 'lat': 34.083589, 'lo...","[{'name': 'Westside Community Adult School', '...","[{'name': 'Maxim', 'lat': 34.081044, 'long': -..."
7,Wedding Channel,34.048681,-118.258778,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Design Digital', 'lat': 34.048574, ...",[{'name': 'Alliance College-Ready Public Schoo...,"[{'name': 'City Club LA', 'lat': 34.050855, 'l..."
8,Causecast,34.018349,-118.486889,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Langley Productions', 'lat': 34.020...","[{'name': 'TeamNow', 'lat': 34.020013, 'long':...","[{'name': 'Disco Eye Wonder Santa M', 'lat': 3..."
9,Mindbox,34.050907,-118.253254,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Please Do Not Enter', 'lat': 34.048...","[{'name': 'Manhattan Prep', 'lat': 34.049836, ...","[{'name': 'City Club LA', 'lat': 34.050855, 'l..."


In [54]:
dfn = df_LosAngeles_copy.copy()
dfu= C_C("Starbucks","starbucks",400,dfn)
dfu.isnull().sum()

Starbucks    11
dtype: int64

In [55]:
df_LosAngeles_copy = pd.concat([df_LosAngeles_copy,dfu],axis=1)
df_LosAngeles_copy.dropna(inplace=True)
df_LosAngeles_copy.reset_index(drop=True,inplace=True)
df_LosAngeles_copy

,name,lat,long,type,Design,school,Clubs,Starbucks
0,Social Platform,34.101617,-118.327071,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'No Such Agency', 'lat': 34.100342, ...","[{'name': 'LCG Footer', 'lat': 34.101301, 'lon...","[{'name': 'H Club Los Angeles', 'lat': 34.1023...","[{'name': 'Starbucks', 'lat': 34.101381, 'long..."
1,DailyStrength,34.043948,-118.468204,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Designers Call', 'lat': 34.045491, ...","[{'name': 'Cotsen Family Foundation', 'lat': 3...","[{'name': 'Water Buffalo Club', 'lat': 34.0439...","[{'name': 'Starbucks', 'lat': 34.044947, 'long..."
2,FASTALLEY,34.081952,-118.357658,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Alex Tailor', 'lat': 34.083589, 'lo...","[{'name': 'Westside Community Adult School', '...","[{'name': 'Maxim', 'lat': 34.081044, 'long': -...","[{'name': 'Starbucks', 'lat': 34.083472, 'long..."
3,Wedding Channel,34.048681,-118.258778,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Design Digital', 'lat': 34.048574, ...",[{'name': 'Alliance College-Ready Public Schoo...,"[{'name': 'City Club LA', 'lat': 34.050855, 'l...","[{'name': 'Starbucks', 'lat': 34.049436, 'long..."
4,Causecast,34.018349,-118.486889,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Langley Productions', 'lat': 34.020...","[{'name': 'TeamNow', 'lat': 34.020013, 'long':...","[{'name': 'Disco Eye Wonder Santa M', 'lat': 3...","[{'name': 'Starbucks', 'lat': 34.017303, 'long..."
5,Mindbox,34.050907,-118.253254,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Please Do Not Enter', 'lat': 34.048...","[{'name': 'Manhattan Prep', 'lat': 34.049836, ...","[{'name': 'City Club LA', 'lat': 34.050855, 'l...","[{'name': 'Starbucks', 'lat': 34.050022, 'long..."
6,TrafficZ,34.037542,-118.441178,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Framestore', 'lat': 34.038156, 'lon...","[{'name': 'Kumon Math and Reading Center', 'la...","[{'name': '4 Play Gentleman's Club', 'lat': 34...","[{'name': 'Starbucks', 'lat': 34.038124, 'long..."
7,SocialVibe,34.081524,-118.382674,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Pacific Design Center', 'lat': 34.0...","[{'name': 'Online Cooking School', 'lat': 34.0...","[{'name': 'Raspoutine Los Angeles', 'lat': 34....","[{'name': 'Starbucks', 'lat': 34.084167, 'long..."
8,Interpolls,34.048573,-118.259570,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Design Digital', 'lat': 34.048574, ...",[{'name': 'Alliance College-Ready Public Schoo...,"[{'name': 'City Club LA', 'lat': 34.050855, 'l...","[{'name': 'Starbucks', 'lat': 34.049436, 'long..."
9,simple season,34.039585,-118.255804,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Europe Fabrics', 'lat': 34.039445, ...","[{'name': 'Andy Kim', 'lat': 34.039286, 'long'...","[{'name': 'The Mayan', 'lat': 34.04053, 'long'...","[{'name': 'Starbucks', 'lat': 34.039737, 'long..."


In [56]:
dfn = df_LosAngeles_copy.copy()
dfu= C_A("Airport","airport",20000,dfn)
dfu.isnull().sum()

Airport    1
dtype: int64

In [57]:
df_LosAngeles_copy = pd.concat([df_LosAngeles_copy,dfu],axis=1)
df_LosAngeles_copy.dropna(inplace=True)
df_LosAngeles_copy.reset_index(drop=True,inplace=True)
df_LosAngeles_copy

,name,lat,long,type,Design,school,Clubs,Starbucks,Airport
0,Social Platform,34.101617,-118.327071,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'No Such Agency', 'lat': 34.100342, ...","[{'name': 'LCG Footer', 'lat': 34.101301, 'lon...","[{'name': 'H Club Los Angeles', 'lat': 34.1023...","[{'name': 'Starbucks', 'lat': 34.101381, 'long...",[{'name': 'Los Angeles International Airport (...
1,DailyStrength,34.043948,-118.468204,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Designers Call', 'lat': 34.045491, ...","[{'name': 'Cotsen Family Foundation', 'lat': 3...","[{'name': 'Water Buffalo Club', 'lat': 34.0439...","[{'name': 'Starbucks', 'lat': 34.044947, 'long...",[{'name': 'Los Angeles International Airport (...
2,FASTALLEY,34.081952,-118.357658,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Alex Tailor', 'lat': 34.083589, 'lo...","[{'name': 'Westside Community Adult School', '...","[{'name': 'Maxim', 'lat': 34.081044, 'long': -...","[{'name': 'Starbucks', 'lat': 34.083472, 'long...",[{'name': 'Los Angeles International Airport (...
3,Wedding Channel,34.048681,-118.258778,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Design Digital', 'lat': 34.048574, ...",[{'name': 'Alliance College-Ready Public Schoo...,"[{'name': 'City Club LA', 'lat': 34.050855, 'l...","[{'name': 'Starbucks', 'lat': 34.049436, 'long...",[{'name': 'Los Angeles International Airport (...
4,Causecast,34.018349,-118.486889,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Langley Productions', 'lat': 34.020...","[{'name': 'TeamNow', 'lat': 34.020013, 'long':...","[{'name': 'Disco Eye Wonder Santa M', 'lat': 3...","[{'name': 'Starbucks', 'lat': 34.017303, 'long...",[{'name': 'Los Angeles International Airport (...
5,Mindbox,34.050907,-118.253254,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Please Do Not Enter', 'lat': 34.048...","[{'name': 'Manhattan Prep', 'lat': 34.049836, ...","[{'name': 'City Club LA', 'lat': 34.050855, 'l...","[{'name': 'Starbucks', 'lat': 34.050022, 'long...",[{'name': 'Los Angeles International Airport (...
6,TrafficZ,34.037542,-118.441178,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Framestore', 'lat': 34.038156, 'lon...","[{'name': 'Kumon Math and Reading Center', 'la...","[{'name': '4 Play Gentleman's Club', 'lat': 34...","[{'name': 'Starbucks', 'lat': 34.038124, 'long...",[{'name': 'Los Angeles International Airport (...
7,SocialVibe,34.081524,-118.382674,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Pacific Design Center', 'lat': 34.0...","[{'name': 'Online Cooking School', 'lat': 34.0...","[{'name': 'Raspoutine Los Angeles', 'lat': 34....","[{'name': 'Starbucks', 'lat': 34.084167, 'long...",[{'name': 'Los Angeles International Airport (...
8,Interpolls,34.048573,-118.259570,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Design Digital', 'lat': 34.048574, ...",[{'name': 'Alliance College-Ready Public Schoo...,"[{'name': 'City Club LA', 'lat': 34.050855, 'l...","[{'name': 'Starbucks', 'lat': 34.049436, 'long...",[{'name': 'Los Angeles International Airport (...
9,simple season,34.039585,-118.255804,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Europe Fabrics', 'lat': 34.039445, ...","[{'name': 'Andy Kim', 'lat': 34.039286, 'long'...","[{'name': 'The Mayan', 'lat': 34.04053, 'long'...","[{'name': 'Starbucks', 'lat': 34.039737, 'long...",[{'name': 'Los Angeles International Airport (...


In [58]:
dfn = df_LosAngeles_copy.copy()
dfu= C_S("Satdium","Pauley Pavilion",10000,dfn)
dfu.isnull().sum()

Satdium    0
dtype: int64

In [59]:
df_LosAngeles_copy = pd.concat([df_LosAngeles_copy,dfu],axis=1)
df_LosAngeles_copy.dropna(inplace=True)
df_LosAngeles_copy.reset_index(drop=True,inplace=True)
df_LosAngeles_copy

,name,lat,long,type,Design,school,Clubs,Starbucks,Airport,Satdium
0,Social Platform,34.101617,-118.327071,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'No Such Agency', 'lat': 34.100342, ...","[{'name': 'LCG Footer', 'lat': 34.101301, 'lon...","[{'name': 'H Club Los Angeles', 'lat': 34.1023...","[{'name': 'Starbucks', 'lat': 34.101381, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pavilions', 'lat': 34.083649, 'long..."
1,DailyStrength,34.043948,-118.468204,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Designers Call', 'lat': 34.045491, ...","[{'name': 'Cotsen Family Foundation', 'lat': 3...","[{'name': 'Water Buffalo Club', 'lat': 34.0439...","[{'name': 'Starbucks', 'lat': 34.044947, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,..."
2,FASTALLEY,34.081952,-118.357658,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Alex Tailor', 'lat': 34.083589, 'lo...","[{'name': 'Westside Community Adult School', '...","[{'name': 'Maxim', 'lat': 34.081044, 'long': -...","[{'name': 'Starbucks', 'lat': 34.083472, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,..."
3,Wedding Channel,34.048681,-118.258778,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Design Digital', 'lat': 34.048574, ...",[{'name': 'Alliance College-Ready Public Schoo...,"[{'name': 'City Club LA', 'lat': 34.050855, 'l...","[{'name': 'Starbucks', 'lat': 34.049436, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Dorothy Chandler Pavilion', 'lat': ..."
4,Causecast,34.018349,-118.486889,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Langley Productions', 'lat': 34.020...","[{'name': 'TeamNow', 'lat': 34.020013, 'long':...","[{'name': 'Disco Eye Wonder Santa M', 'lat': 3...","[{'name': 'Starbucks', 'lat': 34.017303, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,..."
5,Mindbox,34.050907,-118.253254,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Please Do Not Enter', 'lat': 34.048...","[{'name': 'Manhattan Prep', 'lat': 34.049836, ...","[{'name': 'City Club LA', 'lat': 34.050855, 'l...","[{'name': 'Starbucks', 'lat': 34.050022, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Dorothy Chandler Pavilion', 'lat': ..."
6,TrafficZ,34.037542,-118.441178,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Framestore', 'lat': 34.038156, 'lon...","[{'name': 'Kumon Math and Reading Center', 'la...","[{'name': '4 Play Gentleman's Club', 'lat': 34...","[{'name': 'Starbucks', 'lat': 34.038124, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,..."
7,SocialVibe,34.081524,-118.382674,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Pacific Design Center', 'lat': 34.0...","[{'name': 'Online Cooking School', 'lat': 34.0...","[{'name': 'Raspoutine Los Angeles', 'lat': 34....","[{'name': 'Starbucks', 'lat': 34.084167, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,..."
8,Interpolls,34.048573,-118.259570,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Design Digital', 'lat': 34.048574, ...",[{'name': 'Alliance College-Ready Public Schoo...,"[{'name': 'City Club LA', 'lat': 34.050855, 'l...","[{'name': 'Starbucks', 'lat': 34.049436, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Dorothy Chandler Pavilion', 'lat': ..."
9,simple season,34.039585,-118.255804,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Europe Fabrics', 'lat': 34.039445, ...","[{'name': 'Andy Kim', 'lat': 34.039286, 'long'...","[{'name': 'The Mayan', 'lat': 34.04053, 'long'...","[{'name': 'Starbucks', 'lat': 34.039737, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 

In [60]:
dfn = df_LosAngeles_copy.copy()
dfu= C_C("Pet_groomer","Pet grooming",500,dfn)
dfu.isnull().sum()

Pet_groomer    15
dtype: int64

In [61]:
df_LosAngeles_copy = pd.concat([df_LosAngeles_copy,dfu],axis=1)
df_LosAngeles_copy.dropna(inplace=True)
df_LosAngeles_copy.reset_index(drop=True,inplace=True)
df_LosAngeles_copy

,name,lat,long,type,Design,school,Clubs,Starbucks,Airport,Satdium,Pet_groomer
0,Social Platform,34.101617,-118.327071,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'No Such Agency', 'lat': 34.100342, ...","[{'name': 'LCG Footer', 'lat': 34.101301, 'lon...","[{'name': 'H Club Los Angeles', 'lat': 34.1023...","[{'name': 'Starbucks', 'lat': 34.101381, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pavilions', 'lat': 34.083649, 'long...","[{'name': 'Artist's Premium Grooming', 'lat': ..."
1,DailyStrength,34.043948,-118.468204,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Designers Call', 'lat': 34.045491, ...","[{'name': 'Cotsen Family Foundation', 'lat': 3...","[{'name': 'Water Buffalo Club', 'lat': 34.0439...","[{'name': 'Starbucks', 'lat': 34.044947, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...","[{'name': 'Sparky's Pet Grooming', 'lat': 34.0..."
2,TrafficZ,34.037542,-118.441178,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Framestore', 'lat': 34.038156, 'lon...","[{'name': 'Kumon Math and Reading Center', 'la...","[{'name': '4 Play Gentleman's Club', 'lat': 34...","[{'name': 'Starbucks', 'lat': 34.038124, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...","[{'name': 'Lucky Dog Grooming Company', 'lat':..."
3,Attention Span Media,34.062260,-118.343648,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Diamond Foam & Fabric', 'lat': 34.0...",[{'name': 'Citizen Of The World Charter School...,[{'name': 'AAA - Automobile Club of Southern C...,"[{'name': 'Starbucks', 'lat': 34.062483, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...","[{'name': 'Luxury dog grooming', 'lat': 34.062..."
4,National Lampoon,34.097545,-118.368593,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Made At Tbuchmueller', 'lat': 34.09...","[{'name': 'Ds Vocology', 'lat': 34.097735, 'lo...","[{'name': 'Hyde Lounge', 'lat': 34.098211, 'lo...","[{'name': 'Starbucks', 'lat': 34.097697, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...",[{'name': 'Hollywood Hounds Dog Care & Pet Gro...
5,Broken Knuckles Clothing,34.076061,-118.372953,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Maison Bertet', 'lat': 34.075854, '...","[{'name': 'Viva Spanish School', 'lat': 34.076...","[{'name': 'A R Private Club', 'lat': 34.076302...","[{'name': 'Starbucks', 'lat': 34.075122, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...","[{'name': 'Bubbles Mobile Dog Grooming', 'lat'..."
6,Blip'd,34.076061,-118.372953,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Maison Bertet', 'lat': 34.075854, '...","[{'name': 'Viva Spanish School', 'lat': 34.076...","[{'name': 'A R Private Club', 'lat': 34.076302...","[{'name': 'Starbucks', 'lat': 34.075122, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...","[{'name': 'Bubbles Mobile Dog Grooming', 'lat'..."
7,Pipit Interactive,34.045499,-118.470360,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Designers Call', 'lat': 34.045491, ...","[{'name': 'Little Prep Academy', 'lat': 34.046...","[{'name': 'Water Buffalo Club', 'lat': 34.0439...","[{'name': 'Starbucks', 'lat': 34.044947, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...","[{'name': 'Sparky's Pet Grooming', 'lat': 34.0..."


In [62]:
dfn = df_LosAngeles_copy.copy()
dfu= C_C("vegan","vegan_vegetarian",500,dfn)
dfu.isnull().sum()

vegan    4
dtype: int64

In [63]:
df_LosAngeles_copy = pd.concat([df_LosAngeles_copy,dfu],axis=1)
df_LosAngeles_copy.dropna(inplace=True)
df_LosAngeles_copy.reset_index(drop=True,inplace=True)
df_LosAngeles_copy

,name,lat,long,type,Design,school,Clubs,Starbucks,Airport,Satdium,Pet_groomer,vegan
0,Social Platform,34.101617,-118.327071,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'No Such Agency', 'lat': 34.100342, ...","[{'name': 'LCG Footer', 'lat': 34.101301, 'lon...","[{'name': 'H Club Los Angeles', 'lat': 34.1023...","[{'name': 'Starbucks', 'lat': 34.101381, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pavilions', 'lat': 34.083649, 'long...","[{'name': 'Artist's Premium Grooming', 'lat': ...","[{'name': 'Luxe Vegan Food Tours', 'lat': 34.1..."
1,National Lampoon,34.097545,-118.368593,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Made At Tbuchmueller', 'lat': 34.09...","[{'name': 'Ds Vocology', 'lat': 34.097735, 'lo...","[{'name': 'Hyde Lounge', 'lat': 34.098211, 'lo...","[{'name': 'Starbucks', 'lat': 34.097697, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...",[{'name': 'Hollywood Hounds Dog Care & Pet Gro...,[{'name': 'Rosie's Comfort Foods Homestyle Coo...
2,Broken Knuckles Clothing,34.076061,-118.372953,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Maison Bertet', 'lat': 34.075854, '...","[{'name': 'Viva Spanish School', 'lat': 34.076...","[{'name': 'A R Private Club', 'lat': 34.076302...","[{'name': 'Starbucks', 'lat': 34.075122, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...","[{'name': 'Bubbles Mobile Dog Grooming', 'lat'...","[{'name': 'Vegan Glory', 'lat': 34.07623, 'lon..."
3,Blip'd,34.076061,-118.372953,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Maison Bertet', 'lat': 34.075854, '...","[{'name': 'Viva Spanish School', 'lat': 34.076...","[{'name': 'A R Private Club', 'lat': 34.076302...","[{'name': 'Starbucks', 'lat': 34.075122, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...","[{'name': 'Bubbles Mobile Dog Grooming', 'lat'...","[{'name': 'Vegan Glory', 'lat': 34.07623, 'lon..."


In [64]:
#Merge above df into df_LosAngeles_copy


In [65]:
#DROP df_LosAngeles_copy.dropna(subset=['Design'],inplace=True)
df_LosAngeles_copy.reset_index(drop=True, inplace=True)

In [66]:
dfn = df_LosAngeles_copy.copy()
dfu= C_A("Airport","airport",20000,dfn)
dfu.isnull().sum()

Airport    0
dtype: int64

In [67]:
df_LosAngeles_copy = pd.concat([df_LosAngeles_copy,dfu],axis=1)
df_LosAngeles_copy.dropna(inplace=True)
df_LosAngeles_copy.reset_index(drop=True)
df_LosAngeles_copy

,name,lat,long,type,Design,school,Clubs,Starbucks,Airport,Satdium,Pet_groomer,vegan,Airport
0,Social Platform,34.101617,-118.327071,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'No Such Agency', 'lat': 34.100342, ...","[{'name': 'LCG Footer', 'lat': 34.101301, 'lon...","[{'name': 'H Club Los Angeles', 'lat': 34.1023...","[{'name': 'Starbucks', 'lat': 34.101381, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pavilions', 'lat': 34.083649, 'long...","[{'name': 'Artist's Premium Grooming', 'lat': ...","[{'name': 'Luxe Vegan Food Tours', 'lat': 34.1...",[{'name': 'Los Angeles International Airport (...
1,National Lampoon,34.097545,-118.368593,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Made At Tbuchmueller', 'lat': 34.09...","[{'name': 'Ds Vocology', 'lat': 34.097735, 'lo...","[{'name': 'Hyde Lounge', 'lat': 34.098211, 'lo...","[{'name': 'Starbucks', 'lat': 34.097697, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...",[{'name': 'Hollywood Hounds Dog Care & Pet Gro...,[{'name': 'Rosie's Comfort Foods Homestyle Coo...,[{'name': 'Los Angeles International Airport (...
2,Broken Knuckles Clothing,34.076061,-118.372953,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Maison Bertet', 'lat': 34.075854, '...","[{'name': 'Viva Spanish School', 'lat': 34.076...","[{'name': 'A R Private Club', 'lat': 34.076302...","[{'name': 'Starbucks', 'lat': 34.075122, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...","[{'name': 'Bubbles Mobile Dog Grooming', 'lat'...","[{'name': 'Vegan Glory', 'lat': 34.07623, 'lon...",[{'name': 'Los Angeles International Airport (...
3,Blip'd,34.076061,-118.372953,"{'typepoint': {'type': 'Point', 'coordinates':...","[{'name': 'Maison Bertet', 'lat': 34.075854, '...","[{'name': 'Viva Spanish School', 'lat': 34.076...","[{'name': 'A R Private Club', 'lat': 34.076302...","[{'name': 'Starbucks', 'lat': 34.075122, 'long...",[{'name': 'Los Angeles International Airport (...,"[{'name': 'Pauley Pavilion', 'lat': 34.070338,...","[{'name': 'Bubbles Mobile Dog Grooming', 'lat'...","[{'name': 'Vegan Glory', 'lat': 34.07623, 'lon...",[{'name': 'Los Angeles International Airport (...


In [68]:
#design_studio = querySM("Design",[df_LosAngeles_copy['lat'][0],df_LosAngeles_copy['long'][0]], 500)

In [69]:
design_studio = querySM("Design",[34.101617, -118.327071],300)
design_studio

[{'name': 'No Such Agency',
  'lat': 34.100342,
  'long': -118.328955,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.050066, -118.253161]}}}]

In [70]:
school = querySM("school",[34.101617, -118.327071],1000)

In [71]:
airport = queryAirport("LAX",[34.101617, -118.327071],20000)

In [72]:
bar = querySM("bar",[34.101617, -118.327071],500)

In [73]:
starbucks = querySM("coffee",[34.101617, -118.327071],200)

In [74]:
basketball_stadium = queryStadium("Pauley Pavilion",[34.101617, -118.327071],10000)

In [75]:
pet_haircut = querySM("Pet grooming",[34.101617, -118.327071],500)

In [76]:
vegan = querySM("vegan_vegetarian",[34.101617, -118.327071],500)

In [77]:
df_airport = pd.DataFrame(data=airport)
df_bar = pd.DataFrame(data=bar)
df_starbucks = pd.DataFrame(data=starbucks)
df_school = pd.DataFrame(data=school)
df_pet = pd.DataFrame(data=pet_haircut)
df_basketball_stadium = pd.DataFrame(data=basketball_stadium)
df_vegan = pd.DataFrame(data=vegan)
df_design = pd.DataFrame(data=design_studio)

For company 1

In [78]:
company1_lat = 34.101617
company1_long = -118.327071

In [79]:
company1_map = folium.Map(location = [34.101617, -118.327071], zoom_start=10)
folium.Marker([34.101617, -118.327071]).add_to(SM_map)

In [80]:
design_studio1 = querySM("Design",[34.101617, -118.327071 ],300)

In [81]:
school1 = querySM("school",[34.101617, -118.327071 ],1000)

In [82]:
airport1 = queryAirport("LAX",[34.101617, -118.327071 ],20000)

In [83]:
bar1 = querySM("bar",[34.101617, -118.327071 ],500)

In [84]:
starbucks1 = querySM("coffee",[34.101617, -118.327071 ],200)

In [85]:
basketball_stadium1 = queryStadium("Pauley Pavilion",[34.101617, -118.327071 ],10000)

In [86]:
pet_haircut1 = querySM("Pet grooming",[34.101617, -118.327071 ],500)

In [87]:
vegan1 = querySM("vegan_vegetarian",[34.101617, -118.327071 ],500)

In [88]:
df_airport1 = pd.DataFrame(data=airport1)
df_bar1 = pd.DataFrame(data=bar1)
df_starbucks1 = pd.DataFrame(data=starbucks1)
df_school1 = pd.DataFrame(data=school1)
df_pet1 = pd.DataFrame(data=pet_haircut1)
df_basketball_stadium1 = pd.DataFrame(data=basketball_stadium1)
df_vegan1 = pd.DataFrame(data=vegan1)
df_design1 = pd.DataFrame(data=design_studio1)

For company 2

In [89]:
company2_lat = 34.097545
company2_long = -118.368593

In [90]:
company2_map = folium.Map(location = [34.097545, -118.368593], zoom_start=10)
folium.Marker([34.097545, -118.368593]).add_to(SM_map)

In [91]:
design_studio2 = querySM("Design",[34.097545, -118.368593 ],300)

In [92]:
school2 = querySM("school",[34.097545, -118.368593 ],1000)

In [93]:
airport2 = queryAirport("LAX",[34.097545, -118.368593 ],20000)

In [94]:
bar2 = querySM("bar",[34.097545, -118.368593 ],500)

In [95]:
starbucks2 = querySM("coffee",[34.097545, -118.368593 ],200)

In [96]:
basketball_stadium2 = queryStadium("Pauley Pavilion",[34.097545, -118.368593 ],10000)

In [97]:
pet_haircut2 = querySM("Pet grooming",[34.097545, -118.368593 ],500)

In [98]:
vegan2 = querySM("vegan_vegetarian",[34.097545, -118.368593 ],500)

In [99]:
df_airport2 = pd.DataFrame(data=airport2)
df_bar2 = pd.DataFrame(data=bar2)
df_starbucks2 = pd.DataFrame(data=starbucks2)
df_school2 = pd.DataFrame(data=school2)
df_pet2 = pd.DataFrame(data=pet_haircut2)
df_basketball_stadium2 = pd.DataFrame(data=basketball_stadium2)
df_vegan2 = pd.DataFrame(data=vegan2)
df_design2 = pd.DataFrame(data=design_studio2)

For company 3

In [100]:
company3_lat = 34.076061
company3_long = -118.372953

In [101]:
company3_map = folium.Map(location = [34.076061, -118.372953], zoom_start=10)
folium.Marker([34.076061, -118.372953]).add_to(SM_map)

In [102]:
design_studio3 = querySM("Design",[34.076061, -118.372953 ],300)

In [103]:
school3 = querySM("school",[34.076061, -118.372953 ],1000)

In [104]:
airport3 = queryAirport("LAX",[34.076061, -118.372953 ],20000)

In [105]:
bar3 = querySM("bar",[34.076061, -118.372953 ],500)

In [106]:
starbucks3 = querySM("coffee",[34.076061, -118.372953 ],200)

In [107]:
basketball_stadium3 = queryStadium("Pauley Pavilion",[34.076061, -118.372953 ],10000)

In [108]:
pet_haircut3 = querySM("Pet grooming",[34.076061, -118.372953 ],500)

In [109]:
vegan3 = querySM("vegan_vegetarian",[34.076061, -118.372953 ],500)

In [110]:
df_airport3 = pd.DataFrame(data=airport3)
df_bar3 = pd.DataFrame(data=bar3)
df_starbucks3 = pd.DataFrame(data=starbucks3)
df_school3 = pd.DataFrame(data=school3)
df_pet3 = pd.DataFrame(data=pet_haircut3)
df_basketball_stadium3 = pd.DataFrame(data=basketball_stadium3)
df_vegan3 = pd.DataFrame(data=vegan3)
df_design3 = pd.DataFrame(data=design_studio3)

For company 4

In [111]:
company4_lat = 34.076061
company4_long = -118.372953

In [112]:
company4_map = folium.Map(location = [34.076061, -118.372953], zoom_start=10)
folium.Marker([34.076061, -118.372953]).add_to(SM_map)

In [113]:
design_studio4 = querySM("Design",[34.076061, -118.372953 ],300)

In [114]:
school4 = querySM("school",[34.076061, -118.372953 ],1000)

In [115]:
airport4 = queryAirport("LAX",[34.076061, -118.372953 ],20000)

In [116]:
bar4 = querySM("bar",[34.076061, -118.372953 ],500)

In [117]:
starbucks4 = querySM("coffee",[34.076061, -118.372953 ],200)

In [118]:
basketball_stadium4 = queryStadium("Pauley Pavilion",[34.076061, -118.372953 ],10000)

In [119]:
pet_haircut4 = querySM("Pet grooming",[34.076061, -118.372953 ],500)

In [120]:
vegan4 = querySM("vegan_vegetarian",[34.076061, -118.372953 ],500)

In [121]:
df_airport4 = pd.DataFrame(data=airport4)
df_bar4 = pd.DataFrame(data=bar4)
df_starbucks4 = pd.DataFrame(data=starbucks4)
df_school4 = pd.DataFrame(data=school4)
df_pet4 = pd.DataFrame(data=pet_haircut4)
df_basketball_stadium4 = pd.DataFrame(data=basketball_stadium4)
df_vegan4 = pd.DataFrame(data=vegan4)
df_design4 = pd.DataFrame(data=design_studio4)

In [122]:
from folium.plugins import MarkerCluster

In [123]:
school_group1 = folium.FeatureGroup(name="school")

for index, row in df_school1.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="red",
            prefix="fa",
            icon="school",
            icon_color = "white")
    
    school_group1.add_child(Marker(**venue, icon=icon))
    company1_map.add_child(school_group1)

In [124]:
school_group2 = folium.FeatureGroup(name="school")

for index, row in df_school2.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="red",
            prefix="fa",
            icon="school",
            icon_color = "white")
    
    school_group2.add_child(Marker(**venue, icon=icon))
    company2_map.add_child(school_group2)

In [125]:
school_group3 = folium.FeatureGroup(name="school")

for index, row in df_school3.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="red",
            prefix="fa",
            icon="school",
            icon_color = "white")
    
    school_group3.add_child(Marker(**venue, icon=icon))
    company3_map.add_child(school_group3)

In [126]:
school_group4 = folium.FeatureGroup(name="school")

for index, row in df_school4.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="red",
            prefix="fa",
            icon="school",
            icon_color = "white")
    
    school_group4.add_child(Marker(**venue, icon=icon))
    company4_map.add_child(school_group4)

In [127]:
bar_group1 = folium.FeatureGroup(name="Bar")

for index, row in df_bar1.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="black",
            prefix="fa",
            icon="beer",
            icon_color = "white")
    
    bar_group1.add_child(Marker(**venue, icon=icon))
    company1_map.add_child(bar_group1)

In [128]:
bar_group2 = folium.FeatureGroup(name="Bar")

for index, row in df_bar2.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="black",
            prefix="fa",
            icon="beer",
            icon_color = "white")
    
    bar_group2.add_child(Marker(**venue, icon=icon))
    company2_map.add_child(bar_group2)

In [129]:
bar_group3 = folium.FeatureGroup(name="Bar")

for index, row in df_bar3.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="black",
            prefix="fa",
            icon="beer",
            icon_color = "white")
    
    bar_group3.add_child(Marker(**venue, icon=icon))
    company3_map.add_child(bar_group3)

In [130]:
bar_group4 = folium.FeatureGroup(name="Bar")

for index, row in df_bar4.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="black",
            prefix="fa",
            icon="beer",
            icon_color = "white")
    
    bar_group4.add_child(Marker(**venue, icon=icon))
    company4_map.add_child(bar_group4)

In [131]:
design_group1 = folium.FeatureGroup(name="Design Studio")

for index, row in df_design1.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="green",
            prefix="fa",
            icon="desktop",
            icon_color = "white")
    
    design_group1.add_child(Marker(**venue, icon=icon))
    company1_map.add_child(design_group1)

In [132]:
design_group2 = folium.FeatureGroup(name="Design Studio")

for index, row in df_design2.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="green",
            prefix="fa",
            icon="desktop",
            icon_color = "white")
    
    design_group2.add_child(Marker(**venue, icon=icon))
    company2_map.add_child(design_group2)

In [133]:
design_group3 = folium.FeatureGroup(name="Design Studio")

for index, row in df_design3.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="green",
            prefix="fa",
            icon="desktop",
            icon_color = "white")
    
    design_group3.add_child(Marker(**venue, icon=icon))
    company3_map.add_child(design_group3)

In [134]:
design_group4 = folium.FeatureGroup(name="Design Studio")

for index, row in df_design4.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="green",
            prefix="fa",
            icon="desktop",
            icon_color = "white")
    
    design_group4.add_child(Marker(**venue, icon=icon))
    company4_map.add_child(design_group4)

In [135]:
airport_group1 = folium.FeatureGroup(name="Airport")

for index, row in df_airport1.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="purple",
            prefix="fa",
            icon="plane",
            icon_color = "white")
    
    airport_group1.add_child(Marker(**venue, icon=icon))
    company1_map.add_child(airport_group1)

In [136]:
airport_group2 = folium.FeatureGroup(name="Airport")

for index, row in df_airport2.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="purple",
            prefix="fa",
            icon="plane",
            icon_color = "white")
    
    airport_group2.add_child(Marker(**venue, icon=icon))
    company2_map.add_child(airport_group2)

In [137]:
airport_group3 = folium.FeatureGroup(name="Airport")

for index, row in df_airport3.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="purple",
            prefix="fa",
            icon="plane",
            icon_color = "white")
    
    airport_group3.add_child(Marker(**venue, icon=icon))
    company3_map.add_child(airport_group3)

In [138]:
airport_group4 = folium.FeatureGroup(name="Airport")

for index, row in df_airport4.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="purple",
            prefix="fa",
            icon="plane",
            icon_color = "white")
    
    airport_group4.add_child(Marker(**venue, icon=icon))
    company4_map.add_child(airport_group4)

In [139]:
starbucks_group1 = folium.FeatureGroup(name="Starbucks")

for index, row in df_starbucks1.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="darkred",
            prefix="fa",
            icon="coffee",
            icon_color = "white")
    
    starbucks_group1.add_child(Marker(**venue, icon=icon))
    company1_map.add_child(starbucks_group1)

In [140]:
starbucks_group2 = folium.FeatureGroup(name="Starbucks")

for index, row in df_starbucks2.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="darkred",
            prefix="fa",
            icon="coffee",
            icon_color = "white")
    
    starbucks_group2.add_child(Marker(**venue, icon=icon))
    company2_map.add_child(starbucks_group2)

In [141]:
starbucks_group3 = folium.FeatureGroup(name="Starbucks")

for index, row in df_starbucks3.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="darkred",
            prefix="fa",
            icon="coffee",
            icon_color = "white")
    
    starbucks_group3.add_child(Marker(**venue, icon=icon))
    company3_map.add_child(starbucks_group3)

In [142]:
starbucks_group4 = folium.FeatureGroup(name="Starbucks")

for index, row in df_starbucks4.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="darkred",
            prefix="fa",
            icon="coffee",
            icon_color = "white")
    
    starbucks_group4.add_child(Marker(**venue, icon=icon))
    company4_map.add_child(starbucks_group4)

In [143]:
vegan_group1 = folium.FeatureGroup(name="Vegan Food")

for index, row in df_vegan1.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="beige",
            prefix="fa",
            icon="cannabis",
            icon_color = "white")
    
    vegan_group1.add_child(Marker(**venue, icon=icon))
    company1_map.add_child(vegan_group1)

In [144]:
vegan_group2 = folium.FeatureGroup(name="Vegan Food")

for index, row in df_vegan2.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="beige",
            prefix="fa",
            icon="cannabis",
            icon_color = "white")
    
    vegan_group2.add_child(Marker(**venue, icon=icon))
    company2_map.add_child(vegan_group2)

In [145]:
vegan_group3 = folium.FeatureGroup(name="Vegan Food")

for index, row in df_vegan3.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="beige",
            prefix="fa",
            icon="cannabis",
            icon_color = "white")
    
    vegan_group3.add_child(Marker(**venue, icon=icon))
    company3_map.add_child(vegan_group3)

In [146]:
vegan_group4 = folium.FeatureGroup(name="Vegan Food")

for index, row in df_vegan4.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="beige",
            prefix="fa",
            icon="cannabis",
            icon_color = "white")
    
    vegan_group4.add_child(Marker(**venue, icon=icon))
    company4_map.add_child(vegan_group4)

In [147]:
basket_group1 = folium.FeatureGroup(name="Basket")

for index, row in df_basketball_stadium1.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="orange",
            prefix="fa",
            icon="fa-dribbble",
            icon_color = "white")
    
    basket_group1.add_child(Marker(**venue, icon=icon))
    company1_map.add_child(basket_group1)


In [148]:
basket_group2 = folium.FeatureGroup(name="Basket")

for index, row in df_basketball_stadium2.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="orange",
            prefix="fa",
            icon="fa-dribbble",
            icon_color = "white")
    
    basket_group2.add_child(Marker(**venue, icon=icon))
    company2_map.add_child(basket_group2)


In [149]:
basket_group3 = folium.FeatureGroup(name="Basket")

for index, row in df_basketball_stadium3.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="orange",
            prefix="fa",
            icon="fa-dribbble",
            icon_color = "white")
    
    basket_group3.add_child(Marker(**venue, icon=icon))
    company3_map.add_child(basket_group3)


In [150]:
basket_group4 = folium.FeatureGroup(name="Basket")

for index, row in df_basketball_stadium4.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="orange",
            prefix="fa",
            icon="fa-dribbble",
            icon_color = "white")
    
    basket_group4.add_child(Marker(**venue, icon=icon))
    company4_map.add_child(basket_group4)


In [151]:
pet_group1 = folium.FeatureGroup(name="Pet")

for index, row in df_pet1.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="lightgray",
            prefix="fa",
            icon="paw",
            icon_color = "white")
    
    pet_group1.add_child(Marker(**venue, icon=icon))
    company1_map.add_child(pet_group1)


In [152]:
pet_group2 = folium.FeatureGroup(name="Pet")

for index, row in df_pet2.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="lightgray",
            prefix="fa",
            icon="paw",
            icon_color = "white")
    
    pet_group2.add_child(Marker(**venue, icon=icon))
    company2_map.add_child(pet_group2)

In [153]:
pet_group3 = folium.FeatureGroup(name="Pet")

for index, row in df_pet3.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="lightgray",
            prefix="fa",
            icon="paw",
            icon_color = "white")
    
    pet_group3.add_child(Marker(**venue, icon=icon))
    company3_map.add_child(pet_group3)

In [154]:
pet_group4 = folium.FeatureGroup(name="Pet")

for index, row in df_pet4.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="lightgray",
            prefix="fa",
            icon="paw",
            icon_color = "white")
    
    pet_group4.add_child(Marker(**venue, icon=icon))
    company4_map.add_child(pet_group4)

In [155]:
"""offices_group = folium.FeatureGroup(name="offices")

for index, row in df_LosAngeles_copy.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="red",
            prefix="fa",
            icon="buildings",
            icon_color = "white")
    
    offices_group.add_child(Marker(**venue, icon=icon))
    SM_map_updated.add_child(offices_group)"""

'offices_group = folium.FeatureGroup(name="offices")\n\nfor index, row in df_LosAngeles_copy.iterrows():\n    \n    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}\n    \n    icon = Icon (\n            color="red",\n            prefix="fa",\n            icon="buildings",\n            icon_color = "white")\n    \n    offices_group.add_child(Marker(**venue, icon=icon))\n    SM_map_updated.add_child(offices_group)'

In [156]:
folium.LayerControl(collapsed=False).add_to(company1_map)
folium.Marker([company1_lat, company1_long]).add_to(company1_map)

In [157]:
folium.LayerControl(collapsed=False).add_to(company2_map)
folium.Marker([company2_lat, company2_long]).add_to(company2_map)

In [158]:
folium.LayerControl(collapsed=False).add_to(company3_map)
folium.Marker([company3_lat, company3_long]).add_to(company3_map)

In [159]:
folium.LayerControl(collapsed=False).add_to(company4_map)
folium.Marker([company4_lat, company4_long]).add_to(company4_map)

In [160]:
company1_map

In [161]:
company2_map

In [162]:
company3_map

In [163]:
company4_map